In [1]:
import os, glob
import json
import pandas as pd
import numpy as np

from random import randrange
from collections import defaultdict
from functions import load_txt_as_lst, read_alignments

In [2]:
dataset_dir = "/home/craig.car/repos/chiron/chironata/sentence_aligned_texts/"
eval_dicts_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"
align_rsls_dir = "/home/craig.car/repos/chiron/chironata/data/alignments_rslts/"

# Choose randomly from each text's source sentences

In [99]:
ctsurns = []
transl_names_corpus = []
source_sents = []
source_idx = []
vec_sents_corpus = []
vec_idx_corpus = []
retrieval_sents_corpus = []
retrieval_idx_corpus = []
for text_dict_path in glob.iglob(dataset_dir+"*.json"):
    print(f"working on {text_dict_path}")
    ctsurn = os.path.splitext(os.path.basename(text_dict_path))[0]
    ctsurns.append(ctsurn)
    # load text dict (for align preds)
    with open(text_dict_path) as f:
        text_dict = json.load(f)
    # load eval dict (for sentence retrieval preds)
    with open(eval_dicts_dir+ctsurn+".json") as fp:
        eval_dict = json.load(fp)
    src_num_sents = len(text_dict["src_text"].keys())
    # randomly choose an index
    idx = randrange(src_num_sents)
    source_idx.append(idx)
    source_sents.append(text_dict["src_text"][str(idx)])
    # save for all translations
    vec_sents_text = []
    vec_idx_text = []
    # save for all translations
    retrieval_idx_text = []
    retrieval_sents_text = []
    transl_names_text = []
    for key in text_dict.keys():
        if key != "src_text":
            transl_names_text.append(key)
            # get tgt preds by LaBSE-Vecalign
            vec_preds = text_dict[key]["src2tgt_aligns"][str(idx)]
            # save per translation
            vec_sents_transl = []
            vec_idx_transl = []
            retrieval_idx_transl = []
            retrieval_sents_transl = []
            for tgt_idx in vec_preds:
                if tgt_idx == "null":
                    vec_sents_transl.append("null")
                    vec_idx_transl.append("null")
                else:
                    vec_sents_transl.append(text_dict[key]["tgt_text"][str(tgt_idx)])
                    vec_idx_transl.append(tgt_idx)
            # get tgt preds from sentence retrieval
            retrieval_preds = eval_dict[key]["retrieval_preds"][idx]
            for tgt_idx_retr in retrieval_preds:
                retrieval_idx_transl.append(tgt_idx_retr)
                retrieval_sents_transl.append(text_dict[key]["tgt_text"][str(tgt_idx_retr)])
            # save to text-level lists
            vec_sents_text.append(vec_sents_transl)
            vec_idx_text.append(vec_idx_transl)
            retrieval_sents_text.append(retrieval_sents_transl)
            retrieval_idx_text.append(retrieval_idx_transl)
    # save to corpus-level lists
    vec_sents_corpus.append(vec_sents_text)
    vec_idx_corpus.append(vec_idx_text)
    retrieval_sents_corpus.append(retrieval_sents_text)
    retrieval_idx_corpus.append(retrieval_idx_text)
    transl_names_corpus.append(transl_names_text)


working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0001.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0003.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0004.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg002.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg003.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg002.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg003.json
working on

In [101]:
# ctsurns = []
# transl_names = []
# source_sents = []
# source_idx = []
# vec_sents_corpus = []
# vec_idx_corpus = []
# retrieval_sents_corpus = []
# retrieval_idx_corpus = []
manual_eval_df = pd.DataFrame(ctsurns)
manual_eval_df.columns=["ctsurn"]
manual_eval_df["translation_names"] = transl_names_corpus
manual_eval_df["source_sent"] = source_sents
manual_eval_df["source_idx"] = source_idx
manual_eval_df["pred_vecalign"] = vec_sents_corpus
manual_eval_df["pred_vecalign_idx"] = vec_idx_corpus
manual_eval_df["pred_retrieval"] = retrieval_sents_corpus
manual_eval_df["pred_retrieval_idx"] = retrieval_idx_corpus


In [102]:
manual_eval_df.head()

,ctsurn,translation_names,source_sent,source_idx,pred_vecalign,pred_vecalign_idx,pred_retrieval,pred_retrieval_idx
0,urn:cts:greekLit:tlg0001.tlg001,"[apolloniusRhodius_1892, tlg0001.tlg001.opp-en...",τοῦ δὲ πάρος κολεῶν εὐήκεα φάσγανʼ ἑταῖροι ἔστ...,420,"[[Mais, devant le héros se dressèrent ses comp...","[[1056], [935], [680]]","[[τόκοιο λύματα υτλώσαντο., mais la quantité d...","[[11352, 563, 3334, 561, 3765, 323, 7631, 3221..."
1,urn:cts:greekLit:tlg0003.tlg001,"[thucydides_2_1917, thucydides_1_1852, thucydi...","ᾔρετο δὲ τὸ ὕψος τοῦ τείχους μέγα, καὶ τὸ χῶμα...",1059,"[[Denn von einer Macht, die selbst über andere...","[[942], [2261], [1419], [2384, 2385], [967], [...",[[Hält man ihn aber für nichts besser als ande...,"[[1189, 1009, 345, 2100, 1930, 2497, 2234, 209..."
2,urn:cts:greekLit:tlg0004.tlg001,[diogenesLaertius_1853],"πέμπτον εἶδος ῥητορείας, ὅταν τις εὖ λέγῃ καὶ ...",1880,"[[The fifth species of rhetoric, is when any o...","[[3501, 3502]]","[[The fifth species of rhetoric, is when any o...","[[3501, 3578, 3435, 3580, 3499, 3537, 10221, 3..."
3,urn:cts:greekLit:tlg0005.tlg001,"[theocritus_1855, theocritus_1878, theocritus_...",αὐτὰρ ἐγὼ θάμνοισιν ἄφαρ σκιεροῖσιν ἐκρύφθην ἐ...,488,[[er hatte die struppige Mähne Rings mit Morde...,"[[1675, 1676, 1677, 1678], [2424, 2425, 2426, ...","[[Erst am Feuer geglühte, dem Stuhl auf der Ax...","[[1684, 671, 595, 1686, 1625, 1385, 1380, 1680..."
4,urn:cts:greekLit:tlg0005.tlg002,"[theocritus_1855, theocritus_1878, theocritus_...",τοὺς μὲν ἐγὼ δύστηνος ἐμοῖς ἴδον ὀφθαλμοῖσι βα...,63,"[[Die Nöthe, die früher geblüht hat, Hält dir ...","[[2132, 2133, 2134, 2135, 2136, 2137], [3211, ...","[[Die sei von mir gesetzt, doch setz' ich nich...","[[807, 1598, 652, 1032, 1965, 2141, 1681, 804]..."


# Chose randomly from each text's sentences, choosing (randomly) only one translation per text

In [13]:
# load lookup table
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

In [23]:
ctsurns = []
transl_names = []
source_sents_corpus = []
source_idx_corpus = []
vec_sents_corpus = []
vec_idx_corpus = []
retrieval_sents_corpus = []
retrieval_idx_corpus = []
for text_dict_path in glob.iglob(dataset_dir+"*.json"):
    source_sents = []
    source_idx = []
    print(f"working on {text_dict_path}")
    ctsurn = os.path.splitext(os.path.basename(text_dict_path))[0]
    # get translations
    translations = lookup[ctsurn]
    for idx, name in enumerate(translations):
        if name.endswith(".xml"):
            translations[idx] = translations[idx].rstrip(".xml")
    # load text dict (for align preds)
    with open(text_dict_path) as f:
        text_dict = json.load(f)
    # load eval dict (for sentence retrieval preds)
    with open(eval_dicts_dir+ctsurn+".json") as fp:
        eval_dict = json.load(fp)
    src_num_sents = len(text_dict["src_text"].keys())
    # randomly choose an index for sents
    sent_idx = randrange(src_num_sents)
    source_idx.append(sent_idx)
    source_sents.append(text_dict["src_text"][str(sent_idx)])
    # randomly choose an index for translations
    transl_idx = randrange(len(translations))
    transl_name = translations[transl_idx]
    # print(transl_name)
    # save at text-level
    vec_sents_text = []
    vec_idx_text = []
    retrieval_idx_text = []
    retrieval_sents_text = []
    # get tgt preds by LaBSE-Vecalign
    if transl_name in text_dict.keys():
        vec_preds = text_dict[transl_name]["src2tgt_aligns"][str(sent_idx)]
        for tgt_idx in vec_preds:
            if tgt_idx == "null":
                vec_sents_text.append("null")
                vec_idx_text.append("null")
            else:
                vec_sents_text.append(text_dict[transl_name]["tgt_text"][str(tgt_idx)])
                vec_idx_text.append(tgt_idx)
                # add any other src sents included in alignment
                other_src = text_dict[transl_name]["tgt2src_aligns"][str(tgt_idx)]
                # add to src sents
                for src_idx in other_src:
                    source_idx.append(src_idx)
                    source_sents.append(text_dict["src_text"][str(src_idx)])
        # get tgt preds from sentence retrieval
        retrieval_preds = eval_dict[transl_name]["retrieval_preds"][sent_idx]
        for tgt_idx_retr in retrieval_preds:
            retrieval_idx_text.append(tgt_idx_retr)
            retrieval_sents_text.append(text_dict[transl_name]["tgt_text"][str(tgt_idx_retr)])
        
        # make sure source sents and idx are unique
        source_idx = list(set(source_idx))
        source_sents = list(set(source_sents))
        
        # save to corpus-level lists
        vec_sents_corpus.append(vec_sents_text)
        vec_idx_corpus.append(vec_idx_text)
        retrieval_sents_corpus.append(retrieval_sents_text)
        retrieval_idx_corpus.append(retrieval_idx_text)
        source_sents_corpus.append(source_sents)
        source_idx_corpus.append(source_idx)
        ctsurns.append(ctsurn)
        transl_names.append(transl_name)

    else:
        continue
    



working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0001.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0003.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0004.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg002.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0005.tlg003.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg001.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg002.json
working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0006.tlg003.json
working on

In [24]:
# ctsurns = []
# transl_names = []
# source_sents_corpus = []
# source_idx_corpus = []
# vec_sents_corpus = []
# vec_idx_corpus = []
# retrieval_sents_corpus = []
# retrieval_idx_corpus = []

df_manual_eval = pd.DataFrame(ctsurns)
df_manual_eval.columns=["ctsurn"]
df_manual_eval["translation_names"] = transl_names
df_manual_eval["source_sent"] = source_sents_corpus
df_manual_eval["source_idx"] = source_idx_corpus
df_manual_eval["pred_vecalign"] = vec_sents_corpus
df_manual_eval["pred_vecalign_idx"] = vec_idx_corpus
df_manual_eval["pred_retrieval"] = retrieval_sents_corpus
df_manual_eval["pred_retrieval_idx"] = retrieval_idx_corpus


In [25]:
df_manual_eval.head(10)

,ctsurn,translation_names,source_sent,source_idx,pred_vecalign,pred_vecalign_idx,pred_retrieval,pred_retrieval_idx
0,urn:cts:greekLit:tlg0001.tlg001,apolloniusRhodius_1892,[ἤλυθε δʼ ἀφνειὴν προλιπὼν Γυρτῶνα Κόρωνος Και...,[15],"[Certes il était brave, mais il ne surpassait ...","[136, 135]","[Quant aux deux autres, ils étaient nés d’Anti...","[135, 4159, 4651, 11221, 137, 143, 4137, 209]"
1,urn:cts:greekLit:tlg0003.tlg001,thucydides_2_1917,[ὅμως δὲ πόλιν μεγάλην οἰκοῦντας καὶ ἐν ἤθεσιν...,[980],[In dieser Weise wurde der Friede und das Bünd...,[872],"[Die Truppen, nicht nur der gemeine Mann, sond...","[3132, 1404, 1655, 2211, 2601, 969, 2296, 3028]"
2,urn:cts:greekLit:tlg0004.tlg001,diogenesLaertius_1853,[Θέσεις περὶ ψυχῆς α′.],[2517],"[of Propositions relating to the Soul, one;]",[4617],"[of Propositions relating to the Soul, one;, O...","[4617, 10733, 9760, 11778, 11870, 9750, 10722,..."
3,urn:cts:greekLit:tlg0005.tlg001,theocritus_1855,[εὗρον δʼ ἀέναον κρήνην ὑπὸ λισσάδι πέτρῃ ὕδατ...,[357],[Der stand trunken von Schlägen und spuckte da...,"[1431, 1432, 1433, 1434, 1435, 1436, 1437]",[Nicht sank so der Delphin an dem Meeresgestad...,"[2088, 1397, 1352, 2152, 184, 1404, 2050, 1843]"
4,urn:cts:greekLit:tlg0005.tlg002,theocritus_1913,[πρὸς δʼ ἔτι μʼ ἐπτοίησε διὰ γλυκὺν αἰνὸς ὄνει...,[86],"[« Noble fils d’Augias, oui, ton esprit devine...","[3027, 3028, 3029, 3030, 3031, 3032, 3033]",[J’ai déjà tout l’esprit hanté de mille songes...,"[2270, 2279, 3319, 1509, 290, 2757, 2337, 3064]"
5,urn:cts:greekLit:tlg0005.tlg003,theocritus_1855,[Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,[0],"[Nein, bei'm Pan, dem dieß Ufer gehört!, Wann ...","[608, 602, 603, 604, 605, 606, 607]","[Ist sie doch nicht von Stein, vielleicht sie ...","[551, 1808, 2088, 1814, 1164, 1579, 2098, 606]"
6,urn:cts:greekLit:tlg0006.tlg001,euripides_1911,"[ἰδού, καθαρὸν τὸ χεῖλος αἱ τρίχες τέ μου.]",[314],[forbiti son labbra e mustacchi.],[343],"[forbiti son labbra e mustacchi., M’è arrivato...","[343, 103, 15, 179, 412, 166, 205, 236]"
7,urn:cts:greekLit:tlg0006.tlg002,euripides_1_1884,"[κἄπειτα θάλαμον ἐσπεσοῦσα καὶ λέχος, ἐνταῦθα ...",[90],"[Ici, précède-moi, posant mon pied sur un sol ...","[1957, 1958, 1959, 1960, 1961, 1962, 1963]","[Après que le fils d’Akhilleus fut revenu ici,...","[5241, 825, 5719, 1019, 2549, 6782, 341, 2872]"
8,urn:cts:greekLit:tlg0006.tlg003,euripides_1_1884,[οὐδ’ ἂν προσεῖπον οὐδ’ ἂν ἡψάμην χεροῖν.],[155],"[Je ne lui aurais parlé, ni ne l’aurais touché...",[2624],"[ne vous montrez pas à ses yeux, ni n’approche...","[2479, 447, 3029, 5133, 3315, 4915, 2624, 587]"
9,urn:cts:greekLit:tlg0006.tlg004,euripides_1_1884,[δείκνυσι γὰρ κέλευθον ἀσφαλεστάτην· ἐχθρὸς μὲ...,[432],"[Élève en homme Orestès que voici.Embrasse-le,...",[6748],[Il révélera aux ennemis les bêtes féroces cac...,"[1332, 4432, 400, 399, 3940, 4149, 364, 641]"


In [26]:
df_manual_eval.shape

(708, 8)

In [27]:
pathout = "/home/craig.car/repos/chiron/chironata/data/sample_manual_eval.csv"
df_manual_eval.to_csv(pathout, index=False)

# Choose randomly from each text's alignments

In [80]:
ctsurns = []
translation_names = []
aligns = []

source_sents = []
source_idx = []
vec_sents_corpus = []
vec_idx_corpus = []
retrieval_sents_corpus = []
retrieval_idx_corpus = []

for align_path in glob.iglob(align_rsls_dir+"*.rslts"):
    print(f"working on {text_dict_path}")
    ctsurn = os.path.splitext(os.path.basename(align_path))[0].split("_", maxsplit=1)[0]
    transl_name = os.path.splitext(os.path.basename(align_path))[0].split("_", maxsplit=1)[1]
    translation_names.append(transl_name)
    print(ctsurn)
    print(transl_name)
    ctsurns.append(ctsurn)
    # load alignment results
    # align_rslts = load_txt_as_lst(align_path)
    align_rslts = read_alignments(align_path)
    # randomly choose an index
    align_idx = randrange(len(align_rslts))
    alignment = align_rslts[align_idx]
    aligns.append(alignment)
    print(alignment)
    # load text dict
    text_dict_path = dataset_dir+ctsurn+".json"
    with open(text_dict_path) as f:
        text_dict = json.load(f)
    # get src and tgt sents from the alignment
    print(text_dict.keys())
    src_idx_aligns = alignment[0]
    tgt_idx_aligns = alignment[1]
    print(src_idx_aligns)
    print(tgt_idx_aligns)
    break
    
    
    # load eval dict (for sentence retrieval preds)
    with open(eval_dicts_dir+ctsurn+".json") as fp:
        eval_dict = json.load(fp)
    src_num_sents = len(text_dict["src_text"].keys())
    
    
    source_sents.append(text_dict["src_text"][str(idx)])
    # save for all translations
    vec_sents_text = []
    vec_idx_text = []
    # save for all translations
    retrieval_idx_text = []
    retrieval_sents_text = []
    
    for key in text_dict.keys():
        if key != "src_text":
            transl_names_text.append(key)
            # get tgt preds by LaBSE-Vecalign
            vec_preds = text_dict[key]["aligns_idx"][str(idx)]
            # save per translation
            vec_sents_transl = []
            vec_idx_transl = []
            retrieval_idx_transl = []
            retrieval_sents_transl = []
            for tgt_idx in vec_preds:
                if tgt_idx == "null":
                    vec_sents_transl.append("null")
                    vec_idx_transl.append("null")
                else:
                    vec_sents_transl.append(text_dict[key]["tgt_text"][str(tgt_idx)])
                    vec_idx_transl.append(tgt_idx)
            # get tgt preds from sentence retrieval
            retrieval_preds = eval_dict[key]["retrieval_preds"][idx]
            for tgt_idx_retr in retrieval_preds:
                retrieval_idx_transl.append(tgt_idx_retr)
                retrieval_sents_transl.append(text_dict[key]["tgt_text"][str(tgt_idx_retr)])
            # save to text-level lists
            vec_sents_text.append(vec_sents_transl)
            vec_idx_text.append(vec_idx_transl)
            retrieval_sents_text.append(retrieval_sents_transl)
            retrieval_idx_text.append(retrieval_idx_transl)
    # save to corpus-level lists
    vec_sents_corpus.append(vec_sents_text)
    vec_idx_corpus.append(vec_idx_text)
    retrieval_sents_corpus.append(retrieval_sents_text)
    retrieval_idx_corpus.append(retrieval_idx_text)
    transl_names_corpus.append(transl_names_text)

working on /home/craig.car/repos/chiron/chironata/sentence_aligned_texts/urn:cts:greekLit:tlg0001.tlg001.json
urn:cts:greekLit:tlg0001.tlg001
apolloniusRhodius_1892
([], [4126])
dict_keys(['src_text', 'apolloniusRhodius_1_1791', 'tlg0001.tlg001.opp-eng1', 'apolloniusRhodius_1892'])
[]
[4126]
